In [1]:
from google.colab import files
uploaded = files.upload()


Saving rfm_metrics.csv to rfm_metrics.csv


In [2]:
import pandas as pd

df = pd.read_csv("rfm_metrics.csv")
df.head()


,CustomerID,Recency,Frequency,Monetary
0,12346.0,326,1,77183.60
1,12347.0,3,7,4310.00
2,12348.0,76,4,1797.24
3,12349.0,19,1,1757.55
4,12350.0,311,1,334.40


In [3]:
df['R_Score'] = pd.qcut(df['Recency'], 5, labels=[5,4,3,2,1]).astype(int)

In [4]:
df['F_Score'] = pd.qcut(df['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)

In [5]:
df['M_Score'] = pd.qcut(df['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)

In [6]:
df['RFM_Score'] = (
    df['R_Score'].astype(str) +
    df['F_Score'].astype(str) +
    df['M_Score'].astype(str)
)

In [7]:
df.head()

,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
0,12346.0,326,1,77183.60,1,1,5,115
1,12347.0,3,7,4310.00,5,5,5,555
2,12348.0,76,4,1797.24,2,4,4,244
3,12349.0,19,1,1757.55,4,1,4,414
4,12350.0,311,1,334.40,1,1,2,112


In [8]:
def segment_customer(row):
    r, f, m = row['R_Score'], row['F_Score'], row['M_Score']

    if (r >= 4) and (f >= 4) and (m >= 4):
        return 'Champions'
    elif (f >= 4 and m >= 3):
        return 'Loyal Customers'
    elif (r >= 4 and (f == 3 or m == 3)):
        return 'Potential Loyalists'
    elif (r == 5 and f == 1):
        return 'New Customers'
    elif (r == 4 and f == 1):
        return 'Promising'
    elif (r == 2 and f >= 3):
        return 'At Risk'
    elif (r == 1 and f >= 4):
        return 'Cannot Lose Them'
    elif (r == 1 and f == 1):
        return 'Hibernating'
    elif (r == 2 and f == 1):
        return 'Needs Attention'
    elif (m == 5 and r >= 3):
        return 'Big Spenders'
    else:
        return 'Others'

df['Segment'] = df.apply(segment_customer, axis=1)


In [9]:
df['Segment'].value_counts()

,count
Segment,
Others,316
Champions,214
Loyal Customers,160
Hibernating,89
Potential Loyalists,81
At Risk,56
Needs Attention,44
Promising,18
New Customers,10


In [10]:
segment_summary = df.groupby('Segment').agg({
    'CustomerID':'count',
    'Monetary':'sum',
    'Frequency':'mean',
    'Recency':'mean'
}).reset_index()

segment_summary

,Segment,CustomerID,Monetary,Frequency,Recency
0,At Risk,56,53542.39,2.392857,112.285714
1,Big Spenders,9,39923.90,2.000000,39.777778
2,Cannot Lose Them,3,1200.24,4.000000,213.666667
3,Champions,214,1329111.46,12.093458,13.088785
4,Hibernating,89,110291.14,1.000000,282.314607
5,Loyal Customers,160,376712.36,5.025000,59.681250
6,Needs Attention,44,32043.50,1.000000,114.204545
7,New Customers,10,2535.69,1.000000,8.100000
8,Others,316,192336.88,1.563291,126.806962
9,Potential Loyalists,81,70947.48,2.172840,16.345679


In [11]:
df.to_csv("rfm_final.csv", index=False)

In [12]:
from google.colab import files
files.download("rfm_final.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>